In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Reading corpus the text file
with open("/content/metamorphosis_clean.txt", 'r', encoding='utf-8') as myfile:
    mytext = myfile.read()

In [2]:
mytext

'\ufeffOne morning, when Gregor Samsa woke from troubled dreams, he found\nhimself transformed in his bed into a horrible vermin.  He lay on\nhis armour-like back, and if he lifted his head a little he could\nsee his brown belly, slightly domed and divided by arches into stiff\nsections.  The bedding was hardly able to cover it and seemed ready\nto slide off any moment.  His many legs, pitifully thin compared\nwith the size of the rest of him, waved about helplessly as he\nlooked.\n\n"What\'s happened to me?" he thought.  It wasn\'t a dream.  His room,\na proper human room although a little too small, lay peacefully\nbetween its four familiar walls.  A collection of textile samples\nlay spread out on the table - Samsa was a travelling salesman - and\nabove it there hung a picture that he had recently cut out of an\nillustrated magazine and housed in a nice, gilded frame.  It showed\na lady fitted out with a fur hat and fur boa who sat upright,\nraising a heavy fur muff that covered the

In [3]:
mytokenizer = Tokenizer()
mytokenizer.fit_on_texts([mytext])
total_words = len(mytokenizer.word_index) + 1

In [4]:
mytokenizer.word_index

{'the': 1,
 'to': 2,
 'and': 3,
 'he': 4,
 'his': 5,
 'of': 6,
 'was': 7,
 'it': 8,
 'had': 9,
 'in': 10,
 'that': 11,
 'a': 12,
 'as': 13,
 'gregor': 14,
 'with': 15,
 'she': 16,
 'him': 17,
 'her': 18,
 'would': 19,
 'not': 20,
 'but': 21,
 'at': 22,
 'for': 23,
 'they': 24,
 'on': 25,
 'all': 26,
 'room': 27,
 'from': 28,
 'could': 29,
 'be': 30,
 'out': 31,
 'have': 32,
 'if': 33,
 'there': 34,
 'been': 35,
 "gregor's": 36,
 'so': 37,
 'father': 38,
 'sister': 39,
 'this': 40,
 'now': 41,
 'himself': 42,
 'door': 43,
 'then': 44,
 'back': 45,
 'mother': 46,
 'up': 47,
 'even': 48,
 'into': 49,
 'no': 50,
 'did': 51,
 'more': 52,
 'one': 53,
 'their': 54,
 'when': 55,
 'were': 56,
 'what': 57,
 'about': 58,
 'them': 59,
 'way': 60,
 'only': 61,
 'time': 62,
 'i': 63,
 'by': 64,
 'than': 65,
 'you': 66,
 'just': 67,
 'said': 68,
 'little': 69,
 'any': 70,
 'do': 71,
 'get': 72,
 'other': 73,
 'still': 74,
 'first': 75,
 'or': 76,
 'made': 77,
 'go': 78,
 'some': 79,
 'while': 80,
 's

In [5]:
my_input_sequences = []
for line in mytext.split('\n'):
    #print(line)
    token_list = mytokenizer.texts_to_sequences([line])[0]
    #print(token_list)
    for i in range(1, len(token_list)):
        my_n_gram_sequence = token_list[:i+1]
        #print(my_n_gram_sequence)
        my_input_sequences.append(my_n_gram_sequence)
        #print(input_sequences)

In [6]:
max_sequence_len = max([len(seq) for seq in my_input_sequences])
input_sequences = np.array(pad_sequences(my_input_sequences, maxlen=max_sequence_len, padding='pre'))

In [7]:
input_sequences[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0, 1313,  139,   55], dtype=int32)

In [8]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [ ]:
X[1]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 99,  4])

In [9]:
y

array([139,  55,  14, ...,  18, 930, 131], dtype=int32)

In [10]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [11]:
y[1]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [12]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 16, 100)           261800    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 2618)              395318    
                                                                 
Total params: 807718 (3.08 MB)
Trainable params: 807718 (3.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
632/632 [==============================] - 28s 40ms/step - loss: 6.1793 - accuracy: 0.0533
Epoch 2/100
632/632 [==============================] - 25s 40ms/step - loss: 5.7254 - accuracy: 0.0760
Epoch 3/100
632/632 [==============================] - 25s 40ms/step - loss: 5.3089 - accuracy: 0.1157
Epoch 4/100
632/632 [==============================] - 25s 40ms/step - loss: 5.0154 - accuracy: 0.1359
Epoch 5/100
632/632 [==============================] - 26s 41ms/step - loss: 4.7889 - accuracy: 0.1503
Epoch 6/100
632/632 [==============================] - 27s 42ms/step - loss: 4.5865 - accuracy: 0.1612
Epoch 7/100
632/632 [==============================] - 25s 40ms/step - loss: 4.3911 - accuracy: 0.1756
Epoch 8/100
632/632 [==============================] - 25s 40ms/step - loss: 4.1988 - accuracy: 0.1919
Epoch 9/100
632/632 [==============================] - 25s 40ms/step - loss: 4.0091 - accuracy: 0.2081
Epoch 10/100
632/632 [==============================] - 25s 40ms/step - l

In [37]:
input_text = "he had to"
predict_next_words= 3

for _ in range(predict_next_words):
    token_list = mytokenizer.texts_to_sequences([input_text])[0]
    print(token_list)
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in mytokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    input_text += " " + output_word

print(input_text)

[4, 9, 2]
1/1 [==============================] - 0s 35ms/step
[4, 9, 2, 551]
1/1 [==============================] - 0s 39ms/step
[4, 9, 2, 551, 248]
1/1 [==============================] - 0s 39ms/step
he had to wait until midday
